### Code to find goalposts and the ball

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Reshape, Flatten, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import SGD, Adam, RMSprop
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [3]:
import glob
import bisect
from os.path import basename
import xml.etree.ElementTree
import numpy as np
import cv2

In [4]:
def Reduzida25_skip(input_size, cosize):

    input_image = Input(shape=(input_size, cosize, 3))

    # Layer 1
    x = Conv2D(4, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input_image)
    x = BatchNormalization(name='norm_1')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(8, (3,3), strides=(1,1), padding='same', name='conv_' + str(0+2), use_bias=False)(x)
    x = BatchNormalization(name='norm_' + str(0+2))(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    
        # Layer 2 - 5
    for i in range(1,4):
        x = Conv2D(4*(2**i), (3,3), strides=(1,1), padding='same', name='conv_' + str(i+2), use_bias=False)(x)
        x = BatchNormalization(name='norm_' + str(i+2))(x)
        x = LeakyReLU(alpha=0.1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

        # Layer 6
    x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x)
    x = BatchNormalization(name='norm_6')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='same')(x)
    
    skip_connection = x

        # Layer 7 - 8
    for i in range(0,2):
        x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_' + str(i+7), use_bias=False)(x)
        x = BatchNormalization(name='norm_' + str(i+7))(x)
        x = LeakyReLU(alpha=0.1)(x)
        
    skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_skip', use_bias=False)(skip_connection)
    skip_connection = BatchNormalization(name='norm_21')(skip_connection)
    skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
    #skip_connection = Lambda(space_to_depth_x2)(skip_connection)

    x = concatenate([skip_connection, x])
        
        
    x = Conv2D(10, (1,1), strides = (1,1), padding = 'same', name = 'conv_9', use_bias = True)(x)
        
    model = Model(inputs = input_image, outputs=x, name='Fast')
        
    
    return model

In [5]:
def minha2_skip_pequena(input_size, cosize):

    input_image = Input(shape=(input_size, cosize, 3))

    # Layer 1
    #Originalmente era quatro, mas coloquei oito
    x = Conv2D(8, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input_image)
    x = BatchNormalization(name='norm_1')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2D(8*(2**0), (3,3), strides=(1,1), padding='same', name='conv_' + str(2), use_bias=False)(x)
    x = BatchNormalization(name='norm_' + str(2))(x)
    x = LeakyReLU(alpha=0.1)(x)
    
        # Layer 2 - 5
    for i in range(1,4):
        x = Conv2D(8*(2**i), (3,3), strides=(1,1), padding='same', name='conv_' + str(i+2), use_bias=False)(x)
        x = BatchNormalization(name='norm_' + str(i+2))(x)
        x = LeakyReLU(alpha=0.1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

        # Layer 6
    x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x)
    x = BatchNormalization(name='norm_6')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='same')(x)
    
    skip_connection = x

        # Layer 7 - 8
    for i in range(0,2):
        x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_' + str(i+7), use_bias=False)(x)
        x = BatchNormalization(name='norm_' + str(i+7))(x)
        x = LeakyReLU(alpha=0.1)(x)
        
    skip_connection = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_skip', use_bias=False)(skip_connection)
    skip_connection = BatchNormalization(name='norm_21')(skip_connection)
    skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
    #skip_connection = Lambda(space_to_depth_x2)(skip_connection)

    x = concatenate([skip_connection, x])
        
        
    x = Conv2D(10, (1,1), strides = (1,1), padding = 'same', name = 'conv_9', use_bias = True)(x)
        
    model = Model(inputs = input_image, outputs=x, name='Fast')
        
    
    return model

In [6]:
def minha2_skip_pequena2(input_size, cosize):

    input_image = Input(shape=(input_size, cosize, 3))

    # Layer 1
    #Originalmente era quatro, mas coloquei oito
    x = Conv2D(8, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input_image)
    x = BatchNormalization(name='norm_1')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2D(8*(2**0), (3,3), strides=(1,1), padding='same', name='conv_' + str(2), use_bias=False)(x)
    x = BatchNormalization(name='norm_' + str(2))(x)
    x = LeakyReLU(alpha=0.1)(x)
    
        # Layer 2 - 5
    for i in range(1,4):
        x = Conv2D(8*(2**i), (3,3), strides=(1,1), padding='same', name='conv_' + str(i+2), use_bias=False)(x)
        x = BatchNormalization(name='norm_' + str(i+2))(x)
        x = LeakyReLU(alpha=0.1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

        # Layer 6
    x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x)
    x = BatchNormalization(name='norm_6')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='same')(x)
    
    skip_connection = x

        # Layer 7 - 8
    for i in range(0,2):
        x = Conv2D(128*2**(i), (3,3), strides=(1,1), padding='same', name='conv_' + str(i+7), use_bias=False)(x)
        x = BatchNormalization(name='norm_' + str(i+7))(x)
        x = LeakyReLU(alpha=0.1)(x)
        
    skip_connection = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_skip', use_bias=False)(skip_connection)
    skip_connection = BatchNormalization(name='norm_21')(skip_connection)
    skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
    #skip_connection = Lambda(space_to_depth_x2)(skip_connection)

    x = concatenate([skip_connection, x])
        
        
    x = Conv2D(10, (1,1), strides = (1,1), padding = 'same', name = 'conv_9', use_bias = True)(x)
        
    model = Model(inputs = input_image, outputs=x, name='Fast')
        
    
    return model

In [7]:
rede = minha2_skip_pequena(120,160)

In [8]:
pasta_atual = os.getcwd()

In [9]:
def Metodo3(pasta):  #trave com dimensoes
    
    nomesdearquivos = glob.glob(pasta + '/*.xml')
    fotos = glob.glob(pasta + '/*.png')
    maisfotos = glob.glob(pasta+ '/*.jpg')
    for nome in maisfotos:
        fotos.append(nome)
    fotosjpeg = glob.glob(pasta + '/*.jpeg')
    for nome in fotosjpeg:
        fotos.append(nome)
    fotosJPG = glob.glob(pasta + '/*.JPG')
    for nome in fotosJPG:
        fotos.append(nome)
    nomesdearquivos.sort()
    fotos.sort()
    nomesdasanotacoes = list()
    for nomeindo in nomesdearquivos:
        bisect.insort(nomesdasanotacoes, basename(nomeindo).split('.')[0])
        
    
    para_imprimir = list()
    entrada = list()
    for nome in nomesdasanotacoes:
        e = xml.etree.ElementTree.parse(pasta+'/'+nome+'.xml').getroot()
        objects = e.findall('object')           
        
        resposta = np.zeros((15,20,10), dtype=float)
        
        ball = None
        goalpost = None
        goalpost_2 = None
        
        if objects is None:
            para_imprimir.append(resposta)
        else:
            tam = len(objects)
            for i in range(0, tam):
                if objects[i][0].text == 'ball':
                    ball = objects[i]
                elif objects[i][0].text == 'goalpost':
                    goalpost = objects[i]
                elif objects[i][0].text == 'goalpost_2':
                    goalpost_2 = objects[i]
            
            
        if ball is not None:
                
            ball_xmin = float(ball[4][0].text)/4
            ball_ymin = float(ball[4][1].text)/4
            ball_xmax = float(ball[4][2].text)/4
            ball_ymax = float(ball[4][3].text)/4
                
            ball_Xc = (ball_xmin + ball_xmax)/2
            ball_posicx = int((ball_Xc)/8)
            ball_Xc = ball_Xc - ball_posicx*8
            ball_Xc = ball_Xc/8
            ball_Yc = (ball_ymin + ball_ymax)/2
            ball_posicy = int((ball_Yc)/8)
            ball_Yc = ball_Yc - ball_posicy*8
            ball_Yc = ball_Yc/8
            ball_largura = (ball_xmax - ball_xmin)/160
            ball_altura = (ball_ymax - ball_ymin)/160
                
            resposta[ball_posicy][ball_posicx][0:5] = (1, ball_Xc, ball_Yc, ball_largura, ball_altura)
            
                      
        if goalpost is not None:
            
            goalpost_xmin = float(goalpost[4][0].text)/4
            goalpost_ymin = float(goalpost[4][1].text)/4
            goalpost_xmax = float(goalpost[4][2].text)/4
            goalpost_ymax = float(goalpost[4][3].text)/4
            
            goalpost_Xc = (goalpost_xmin + goalpost_xmax)/2
            goalpost_posicx = int((goalpost_Xc)/8)
            goalpost_Xc = goalpost_Xc - goalpost_posicx*8
            goalpost_Xc = goalpost_Xc/8
            goalpost_Yc = (goalpost_ymin + goalpost_ymax)/2
            goalpost_posicy = int((goalpost_Yc)/8)
            goalpost_Yc = goalpost_Yc - goalpost_posicy*8
            goalpost_Yc = goalpost_Yc/8
            goalpost_largura = (goalpost_xmax - goalpost_xmin)/160
            goalpost_altura = (goalpost_ymax - goalpost_ymin)/160
            
            if goalpost_posicy > 20:
                print(nome)
                print('y')
            elif goalpost_posicx > 20:
                print(nome)
                print('x')
            
            resposta[goalpost_posicy][goalpost_posicx][5:10] = (1,goalpost_Xc, goalpost_Yc, goalpost_largura, goalpost_altura)
            #Funcao de custo esta errada, pois assim deu certo. Alem disso, escolher outra forma de 
            #marcar a trave
        
        if goalpost_2 is not None:
            
            goalpost2_xmin = float(goalpost_2[4][0].text)/4
            goalpost2_ymin = float(goalpost_2[4][1].text)/4
            goalpost2_xmax = float(goalpost_2[4][2].text)/4
            goalpost2_ymax = float(goalpost_2[4][3].text)/4
            
            goalpost2_Xc = (goalpost2_xmin + goalpost2_xmax)/2
            goalpost2_posicx = int((goalpost2_Xc)/8)
            goalpost2_Xc = goalpost2_Xc - goalpost2_posicx*8
            goalpost2_Xc = goalpost2_Xc/8
            goalpost2_Yc = (goalpost2_ymin + goalpost2_ymax)/2
            goalpost2_posicy = int((goalpost2_Yc)/8)
            goalpost2_Yc = goalpost2_Yc - goalpost2_posicy*8
            goalpost2_Yc = goalpost2_Yc/8
            goalpost2_largura = (goalpost2_xmax - goalpost2_xmin)/160
            goalpost2_altura = (goalpost2_ymax - goalpost2_ymin)/160
        
            resposta[goalpost2_posicy][goalpost2_posicx][5:10] = (1, goalpost2_Xc, goalpost2_Yc, goalpost2_largura, goalpost2_altura)
            #Funcao de custo esta errada, pois assim deu certo. Alem disso, escolher outra forma de 
            #marcar a trave
        
        para_imprimir.append(resposta)
        
    Y = np.array(para_imprimir)

    for arquivo in fotos:
        img = cv2.imread(arquivo)
        img = cv2.resize(img, (160,120), interpolation = cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        entrada.append(img)
        
    
    X = np.array(entrada)
    X = X.astype(float)
    X = X/255
    
    return X, Y


In [10]:
carregar = list()
carregar = os.listdir(pasta_atual + '/Conjunto de Treino 2019')

In [11]:
x, y = Metodo3(pasta_atual +'/Conjunto de Treino 2019/' + carregar[len(carregar)-1])
carregar.pop();

In [12]:
for pasta in carregar:
    x1, y1 = Metodo3(pasta_atual +'/Conjunto de Treino 2019/' + pasta)
    x = np.concatenate((x,x1), axis=0)
    y = np.concatenate((y,y1), axis=0)
    x1 = None
    y1 = None

In [13]:
x_val, y_val = Metodo3(pasta_atual +'/Validation set')

In [14]:
print("Formato do X", x.shape)
print("Foramto do Y", y.shape)

('Formato do X', (14934, 120, 160, 3))
('Foramto do Y', (14934, 15, 20, 10))


In [15]:
print("Formato do X_val", x_val.shape)
print("Foramto do Y_val", y_val.shape)

('Formato do X_val', (1173, 120, 160, 3))
('Foramto do Y_val', (1173, 15, 20, 10))


In [16]:
def ball_goalpost_loss3(y_true, y_pred): #Trave com dimensoes
    
    #BALL
    Xc_true = y_true[...,1]
    Xc_pred = tf.sigmoid(y_pred[..., 1])        ##mudei aqui
    Yc_true = y_true[..., 2]
    Yc_pred = tf.sigmoid(y_pred[..., 2])        ##mudei aqui
    W_true = tf.sqrt(tf.abs(y_true[...,3]))
    H_true = tf.sqrt(tf.abs(y_true[...,4]))
    WH_pred = tf.sqrt(tf.abs(tf.exp(y_pred[..., 3:5])*np.reshape(ANCHORS, [1,1,BOX,2])))
    W_pred = WH_pred[...,0]
    H_pred = WH_pred[...,1]
    confidence_true = y_true[...,0]
    confidence_pred = tf.sigmoid(y_pred[...,0])
    class_true = y_true[...,0]
    
    #GOALPOST
    Xc_true_post = y_true[...,6]
    Xc_pred_post = tf.sigmoid(y_pred[..., 6])        ##mudei aqui
    Yc_true_post = y_true[..., 7]
    Yc_pred_post = tf.sigmoid(y_pred[..., 7])        ##mudei aqui
    W_true_post = tf.sqrt(tf.abs(y_true[...,8]))
    H_true_post = tf.sqrt(tf.abs(y_true[...,9]))
    WH_pred_post = tf.sqrt(tf.abs(tf.exp(y_pred[..., 8:10])*np.reshape(ANCHORS_post, [1,1,BOX,2])))
    W_pred_post = WH_pred_post[...,0]
    H_pred_post = WH_pred_post[...,1]
    confidence_true_post = y_true[...,5]
    confidence_pred_post = tf.sigmoid(y_pred[...,5])
    class_true_post = y_true[...,5]
    
    
    #BALL
    Xc_pred = tf.multiply(class_true,Xc_pred)
    Yc_pred = tf.multiply(class_true, Yc_pred)
    H_pred = tf.multiply(class_true, H_pred)
    W_pred = tf.multiply(class_true, W_pred)
    
    
    #GOALPOST
    Xc_pred_post = tf.multiply(class_true_post, Xc_pred_post)
    Yc_pred_post = tf.multiply(class_true_post, Yc_pred_post)
    H_pred_post = tf.multiply(class_true_post, H_pred_post)
    W_pred_post = tf.multiply(class_true_post, W_pred_post)
    
    #BALL
    loss_xy = (tf.square(Xc_pred - Xc_true) + tf.square(Yc_pred - Yc_true))
    loss_xy = 5*tf.reduce_sum(loss_xy)
    
    #GOALPOST    
    loss_xy_post = (tf.square(Xc_pred_post - Xc_true_post) + tf.square(Yc_pred_post - Yc_true_post))
    loss_xy_post = 5*tf.reduce_sum(loss_xy_post)
    
    #BALL
    loss_wh = (tf.square(W_true - W_pred) + tf.square(H_pred - H_true))
    loss_wh = 20*tf.reduce_sum(loss_wh)  #era 10
    
    #GOALPOST
    loss_wh_post = (tf.square(W_true_post - W_pred_post) + tf.square(H_pred_post - H_true_post))
    loss_wh_post = 20*tf.reduce_sum(loss_wh_post)  #era 10
    
    #BALL
    no_object = 1 - class_true
    diff_conf = tf.square(confidence_true - confidence_pred)
    loss_no_object = (tf.multiply(diff_conf, no_object))
    loss_no_object = 10*tf.reduce_sum(loss_no_object)  #era 0.5
    
    loss_object = 50*tf.reduce_sum(tf.multiply(class_true, diff_conf)) #era 5
    
    #GOALPOST
    no_object_post = 1 - class_true_post
    diff_conf_post = tf.square(confidence_true_post - confidence_pred_post)
    loss_no_object_post = (tf.multiply(diff_conf_post, no_object_post))
    loss_no_object_post = 10*tf.reduce_sum(loss_no_object_post)  #era 0.5
    
    loss_object_post = 50*tf.reduce_sum(tf.multiply(class_true_post, diff_conf_post)) #era 5
    
    
    
    #loss_class = tf.nn.sigmoid_cross_entropy_with_logits(labels = class_true, logits = class_pred)
    #loss_class = tf.reduce_sum(loss_class)
    
    loss = loss_xy + loss_wh + loss_no_object + loss_object + loss_xy_post + loss_wh_post + loss_no_object_post + loss_object_post #loss_class
    
    
    return loss

In [17]:
optimizer = Adam (lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optimizer = SGD(lr=1e-4, decay=0.0005, momentum=0.9)
#optimizer = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)
from keras.losses import mean_squared_error

ANCHORS = [5,5]
BOX = 1
ANCHORS_post = [2,5] ##MUDAR AQUI PARA [2,5]


rede.compile(loss=ball_goalpost_loss3, optimizer=optimizer)


rede.fit(x, y, epochs=100, batch_size=5)
#existe tambem o validation split
#validantion_data = (x_val,y_val)
rede.compile(loss=mean_squared_error, optimizer='sgd', metrics = ['accuracy'])
rede.save(pasta_atual+ '/RedeGrande_colfax.hdf5')

Epoch 1/100
14934/14934 [==============================] - 293s 20ms/step - loss: 609.9507
Epoch 2/100
14934/14934 [==============================] - 280s 19ms/step - loss: 173.8215
Epoch 3/100
14934/14934 [==============================] - 279s 19ms/step - loss: 115.0956
Epoch 4/100
14934/14934 [==============================] - 240s 16ms/step - loss: 89.2154
Epoch 5/100
14934/14934 [==============================] - 246s 17ms/step - loss: 75.3102
Epoch 6/100
14934/14934 [==============================] - 266s 18ms/step - loss: 65.9104
Epoch 7/100
14934/14934 [==============================] - 267s 18ms/step - loss: 59.0194
Epoch 8/100
14934/14934 [==============================] - 268s 18ms/step - loss: 54.1385
Epoch 9/100
14934/14934 [==============================] - 268s 18ms/step - loss: 49.6115
Epoch 10/100
14934/14934 [==============================] - 273s 18ms/step - loss: 46.0694
Epoch 11/100
14934/14934 [==============================] - 273s 18ms/step - loss: 43.1292
Epoch

14934/14934 [==============================] - 252s 17ms/step - loss: 7.3770
Epoch 92/100
14934/14934 [==============================] - 256s 17ms/step - loss: 7.5391
Epoch 93/100
14934/14934 [==============================] - 256s 17ms/step - loss: 7.4595
Epoch 94/100
14934/14934 [==============================] - 256s 17ms/step - loss: 7.3414
Epoch 95/100
14934/14934 [==============================] - 267s 18ms/step - loss: 7.2306
Epoch 96/100
14934/14934 [==============================] - 228s 15ms/step - loss: 7.2014
Epoch 97/100
14934/14934 [==============================] - 237s 16ms/step - loss: 7.1917
Epoch 98/100
14934/14934 [==============================] - 251s 17ms/step - loss: 7.0563
Epoch 99/100
14934/14934 [==============================] - 258s 17ms/step - loss: 6.9517
Epoch 100/100
14934/14934 [==============================] - 258s 17ms/step - loss: 6.9994


In [18]:
rede = Reduzida25_skip(120,160)

In [19]:
optimizer = Adam (lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optimizer = SGD(lr=1e-4, decay=0.0005, momentum=0.9)
#optimizer = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)
from keras.losses import mean_squared_error

ANCHORS = [5,5]
BOX = 1
ANCHORS_post = [2,5] ##MUDAR AQUI PARA [2,5]


rede.compile(loss=ball_goalpost_loss3, optimizer=optimizer)


rede.fit(x, y, validation_data = (x_val, y_val), epochs=100, batch_size=5)
#existe tambem o validation split
#validantion_data = (x_val,y_val)
rede.compile(loss=mean_squared_error, optimizer='sgd', metrics = ['accuracy'])
rede.save(pasta_atual+ '/Redepequena_colfax.hdf5')

Train on 14934 samples, validate on 1173 samples
Epoch 1/100
14934/14934 [==============================] - 162s 11ms/step - loss: 1052.0101 - val_loss: 280.0253
Epoch 2/100
14934/14934 [==============================] - 173s 12ms/step - loss: 221.3559 - val_loss: 182.1973
Epoch 3/100
14934/14934 [==============================] - 178s 12ms/step - loss: 148.8932 - val_loss: 148.7764
Epoch 4/100
14934/14934 [==============================] - 177s 12ms/step - loss: 117.8969 - val_loss: 137.3586
Epoch 5/100
14934/14934 [==============================] - 207s 14ms/step - loss: 101.9389 - val_loss: 127.8354
Epoch 6/100
14934/14934 [==============================] - 173s 12ms/step - loss: 90.3267 - val_loss: 122.8448
Epoch 7/100
14934/14934 [==============================] - 181s 12ms/step - loss: 83.3346 - val_loss: 114.6787
Epoch 8/100
14934/14934 [==============================] - 206s 14ms/step - loss: 76.8126 - val_loss: 110.9939
Epoch 9/100
14934/14934 [==============================] 

14934/14934 [==============================] - 189s 13ms/step - loss: 14.8337 - val_loss: 94.5552
Epoch 75/100
14934/14934 [==============================] - 181s 12ms/step - loss: 14.3468 - val_loss: 91.9334
Epoch 76/100
14934/14934 [==============================] - 183s 12ms/step - loss: 14.3989 - val_loss: 91.0028
Epoch 77/100
14934/14934 [==============================] - 230s 15ms/step - loss: 13.9169 - val_loss: 94.3424
Epoch 78/100
14934/14934 [==============================] - 175s 12ms/step - loss: 13.9364 - val_loss: 89.3069
Epoch 79/100
14934/14934 [==============================] - 164s 11ms/step - loss: 13.7734 - val_loss: 91.3889
Epoch 80/100
14934/14934 [==============================] - 209s 14ms/step - loss: 13.6716 - val_loss: 91.1616
Epoch 81/100
14934/14934 [==============================] - 176s 12ms/step - loss: 13.5292 - val_loss: 89.6739
Epoch 82/100
14934/14934 [==============================] - 241s 16ms/step - loss: 13.1173 - val_loss: 93.2396
Epoch 83/100
1

In [20]:
rede = minha2_skip_pequena2(120,160)

In [21]:
optimizer = Adam (lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optimizer = SGD(lr=1e-4, decay=0.0005, momentum=0.9)
#optimizer = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)
from keras.losses import mean_squared_error

ANCHORS = [5,5]
BOX = 1
ANCHORS_post = [2,5] ##MUDAR AQUI PARA [2,5]


rede.compile(loss=ball_goalpost_loss3, optimizer=optimizer)


rede.fit(x, y, validation_data = (x_val, y_val), epochs=100, batch_size=5)
#existe tambem o validation split
#validantion_data = (x_val,y_val)
rede.compile(loss=mean_squared_error, optimizer='sgd', metrics = ['accuracy'])
rede.save(pasta_atual+ '/Rededeteste_colfax.hdf5')

Train on 14934 samples, validate on 1173 samples
Epoch 1/100
14934/14934 [==============================] - 263s 18ms/step - loss: 782.8944 - val_loss: 231.8965
Epoch 2/100
14934/14934 [==============================] - 300s 20ms/step - loss: 196.7309 - val_loss: 166.4851
Epoch 3/100
14934/14934 [==============================] - 319s 21ms/step - loss: 125.6179 - val_loss: 134.9971
Epoch 4/100
14934/14934 [==============================] - 305s 20ms/step - loss: 92.6767 - val_loss: 116.5461
Epoch 5/100
14934/14934 [==============================] - 333s 22ms/step - loss: 76.1336 - val_loss: 107.6614
Epoch 6/100
14934/14934 [==============================] - 296s 20ms/step - loss: 65.0209 - val_loss: 99.4285
Epoch 7/100
14934/14934 [==============================] - 287s 19ms/step - loss: 57.4418 - val_loss: 101.1006
Epoch 8/100
14934/14934 [==============================] - 329s 22ms/step - loss: 52.1071 - val_loss: 101.3221
Epoch 9/100
14934/14934 [==============================] - 33

14934/14934 [==============================] - 283s 19ms/step - loss: 9.6612 - val_loss: 94.6527
Epoch 75/100
14934/14934 [==============================] - 284s 19ms/step - loss: 9.3423 - val_loss: 92.2822
Epoch 76/100
14934/14934 [==============================] - 313s 21ms/step - loss: 9.6145 - val_loss: 91.7640
Epoch 77/100
14934/14934 [==============================] - 300s 20ms/step - loss: 9.2867 - val_loss: 97.0665
Epoch 78/100
14934/14934 [==============================] - 313s 21ms/step - loss: 8.9622 - val_loss: 90.9921
Epoch 79/100
14934/14934 [==============================] - 306s 21ms/step - loss: 9.0051 - val_loss: 86.7717
Epoch 80/100
14934/14934 [==============================] - 298s 20ms/step - loss: 9.2498 - val_loss: 88.6032
Epoch 81/100
14934/14934 [==============================] - 321s 22ms/step - loss: 8.4676 - val_loss: 92.7182
Epoch 82/100
14934/14934 [==============================] - 399s 27ms/step - loss: 8.8125 - val_loss: 91.1300
Epoch 83/100
14934/1493